In [6]:
%pip install pythautomata


[notice] A new release of pip available: 22.1.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Creating a moore machine

#### Dependencies

In [17]:
from pythautomata.base_types.alphabet import Alphabet
from pythautomata.base_types.moore_state import MooreState
from pythautomata.base_types.symbol import SymbolStr
from pythautomata.automata.moore_machine_automaton import MooreMachineAutomaton
from pythautomata.model_comparators.moore_machine_comparison_strategy import MooreMachineComparisonStrategy as ComparisonStrategy
from pythautomata.model_exporters.dot_exporting_mm_strategy import DotExportingMMStrategy

#### Implementation

In [19]:
# First we create an input alphabet with symbols a, b and c
input_alphabet = Alphabet(frozenset((SymbolStr('a'), SymbolStr('b'), SymbolStr('c'))))
# Then the output alphabet with symbols 0 and 1
output_alphabet = Alphabet(frozenset((SymbolStr('0'), SymbolStr('1'))))

a = input_alphabet['a']
b = input_alphabet['b']
c = input_alphabet['c']

# Defining moore states
stateA = MooreState(name="State A", value=output_alphabet['0'])
stateB = MooreState("State B", output_alphabet['1'])
stateC = MooreState("State C", output_alphabet['1'])

# Adding transitions for the previous states
stateA.add_transition(a, stateA)
stateA.add_transition(b, stateB)
stateB.add_transition(a, stateC)
stateB.add_transition(c, stateB)

# Finaly we create the moore machine with the alphabets and states previously created 
moore_machine = MooreMachineAutomaton(input_alphabet, output_alphabet, initial_state=stateA, states=set([stateA, stateB, stateC]), comparator=ComparisonStrategy, name="moore machine with 3 states", exportingStrategies=[DotExportingMMStrategy()])

# Moore machine is exported in the directory ./output_models/moore_machines/ with .dot extension
moore_machine.export('./output_models/moore_machines/')

### Moore machine generator

#### Dependencies

In [20]:
from pythautomata.utilities.nicaud_mm_generator import generate_moore_machine

#### Implementation

In [26]:
# Number of states
number_of_states = 90

generated_moore_machine = generate_moore_machine(input_alphabet, output_alphabet, number_of_states, seed=100)
# The seed is an optional param

# Moore machine is exported in the directory ./output_models/moore_machines/
generated_moore_machine.export('./output_models/moore_machines/')

In [ ]:
# Generate a random Moore Machine with 500 states
input_alphabet = Alphabet.from_strings(["a", "b", "c"])
output_alphabet = Alphabet.from_strings(['0', '1'])
seed = 100
number_of_states = 500
automata = generate_moore_machine(input_alphabet, output_alphabet, number_of_states, seed)

### DFA - moore machine comparator and converter

#### Dependencies

In [28]:
from pythautomata.model_comparators.moore_machine_comparison_strategy import MooreMachineComparisonStrategy
from pythautomata.utilities.automata_converter import AutomataConverter
from pythautomata.automata_definitions.tomitas_grammars import TomitasGrammars
from pythautomata.automata_definitions.sample_moore_machines import SampleMooreMachines

#### Implementation

In [29]:
# Tomitas DFA example
tomitas = TomitasGrammars.get_automaton_1()

# Convert tomitas DFA to moore machine
converted_moore_machine = AutomataConverter.convert_dfa_to_moore_machine(tomitas)

# Manually created tomitas moore machine
tomitas_moore_machine = SampleMooreMachines.get_tomitas_automaton_1()

# Comparate tomitas automatons
converted_moore_machine.__eq__(tomitas_moore_machine)

True

In [19]:
%pip install pymodelextractor

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Moore machine L*

#### Dependencies

In [30]:
from pythautomata.model_comparators.moore_machine_comparison_strategy import MooreMachineComparisonStrategy
from pymodelextractor.learners.observation_table_learners.mm_lstar_learner import MMLStarLearner as MooreMachineLearner
from pymodelextractor.teachers.moore_machines_teacher import MooreMachineTeacher as MMTeacher
from pythautomata.automata_definitions.sample_moore_machines import SampleMooreMachines

#### Implementation

In [31]:
# Create the Moore Machine Learner that implements the L* algorithm
learner = MooreMachineLearner()

# Import a sample Moore Machine
moore_machine = SampleMooreMachines.get_tomitas_automaton_1()

# Create the teacher for the Moore Machine Learner
mm_teacher = MMTeacher(moore_machine, MooreMachineComparisonStrategy())

# Learn the generated Moore Machine
teacher = mm_teacher
result = learner.learn(teacher, verbose=True) 
# verbose=True prints the learning process, the default value is False


# Show that the learned Moore Machine is equivalent to the generated Moore Machine
print("Are equal: ", MooreMachineComparisonStrategy().are_equivalent(
    result.model, moore_machine))

# Export the learned Moore Machine
result.model.export('./output_models/mm_lstar/')

**** Started moore machines lstar ****
 # Starting iteration 1
    . Closed table in 0.0s
    + Made table consistent in 0.0s
    - Made equivalence query in 0.0s
  # Iteration 1 ended, duration: 0.0s
**** Learning finished in 0.0s using 0 counterexamples & final model ended with 2 states ****

Are equal:  True
Unexpected exception when exporting Moore Machine - 024be03be37247f79e0504e43eaf0a76: <class 'graphviz.backend.execute.ExecutableNotFound'>
